In [4]:
import pandas as pd
import numpy as np
import os
import glob as gb
import sys
sys.path.insert(0, "../../")
from clases.bd.conexion2 import MyDatabase2
conn = MyDatabase2()

Conexion exitosa con la Base de datos:irvin_hisminsa


In [5]:
#consulta a la base de datos altitud de establecimiento 
def consulta_rvalidacion(conn,grupo):
    paquete=conn.df("""SELECT * FROM e_vida.reglas_validacion rv WHERE st=1 AND id=%s """ % grupo)   
    return paquete

def consulta_esquema(conn,id):
    paquete=conn.df("""SELECT * FROM e_vida.esquema e WHERE  st=1 AND id_reglas_validacion=%s ORDER BY e.id"""%id)   
    return paquete 

def consulta_codigo(conn,id):
    paquete=conn.df("""SELECT * FROM e_vida.codigos e WHERE  st=1 AND id_esquema=%s"""%id)   
    return paquete 

def consulta_ultima_atencion(conn):
    paquete=conn.df("""WITH atenciones_ordenadas AS (  SELECT mhtd.abrev_tipo_doc, mp.numero_documento, nt.fecha_atencion, nt.id_establecimiento, ROW_NUMBER() OVER (PARTITION BY mp.id_tipo_documento, mp.numero_documento
ORDER BY nt.fecha_atencion DESC) AS rn
FROM maestros.nominal_trama2 nt
INNER JOIN maestros.maestro_paciente mp ON mp.id_paciente = nt.id_paciente
INNER JOIN maestros.maestro_his_tipo_doc mhtd ON mhtd.id_tipo_documento =mp.id_tipo_documento 
)
SELECT abrev_tipo_doc, numero_documento, id_establecimiento
FROM atenciones_ordenadas
WHERE rn = 1;""")   
    return paquete 
u_atencion=consulta_ultima_atencion(conn)

def consulta_pacientes(conn):
    paquete=conn.df(""" SELECT mp.id_paciente ,mhtd.abrev_tipo_doc,mp.numero_documento,mp.genero, mp.fecha_nacimiento,mp.id_pais   FROM maestros.maestro_paciente mp 
INNER JOIN maestros.maestro_his_tipo_doc mhtd ON mp.id_tipo_documento =mhtd.id_tipo_documento  """)   
    paquete['id_paciente'] = paquete['id_paciente']
    return paquete
#paciente_df =consulta_pacientes(conn)

def consulta_eess(conn):
    eess=conn.df(""" SELECT egc.id_eess, egc.nombre_eess, egc.red ,egc.microred ,egc.provincia, egc.distrito  FROM maestros.eess_geresa_cusco egc  """)   
    eess = eess.rename(columns={'id_eess': 'id_establecimiento'})
    return eess
eess_df =consulta_eess(conn)

d:\Irvin\Irvin\Python\pandas\modulos\adolescente\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\pandas\modulos\adolescente\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


In [7]:
data = {'abrev_tipo_doc': [],
        'numero_documento': [],
        'genero': [],
        'fecha_nacimiento': [],
        'edad': [] }
df = pd.DataFrame(data)
df_esquema = pd.DataFrame({})

df_rvalidacion = consulta_rvalidacion(conn,1)

for index, row in df_rvalidacion.iterrows():  
    id_p = row['id']
    inicio = row['inicio']
    fin = row['fin']

     #creamos una lista vacia para almacenas los id
    id_col= []

    #print(id_p)
    df_esquema=consulta_esquema(conn,id_p)

    for h, row1 in df_esquema.iterrows():
        id_es = row1['id']
        cantidad_es = row1['cantidad']
        operador=row1['operador']
        head_esquema=[]
        #crea nuevo data frama pequeño 
        df_min = pd.DataFrame(data)
        #consulta SQL para jalar los codigos 
        df_codigo= consulta_codigo(conn,id_es)
        

        for i, row2 in df_codigo.iterrows():
            dx = row2['codigo_item']
            tdx = row2['tipo_diagnostico']
            lab1="AND nt.lab1='"+row2['lab_1']+"'" if row2['lab_1'] is not None  else ''
            lab2="AND nt.lab2='"+row2['lab_2']+"'" if row2['lab_2'] is not None  else ''
            lab3="AND nt.lab3='"+row2['lab_3']+"'" if row2['lab_3'] is not None  else '' 
            #print(1)

            sql = """  SELECT DISTINCT  mhtd.abrev_tipo_doc, mp.numero_documento,mp.genero,mp.fecha_nacimiento, TO_CHAR(AGE(NOW(), fecha_nacimiento), 'YY "años"') AS edad  ,nt.fecha_atencion  FROM maestros.nominal_trama2 nt 
 INNER JOIN maestros.maestro_paciente mp ON mp.id_paciente =nt.id_paciente 
INNER JOIN maestros.maestro_his_tipo_doc mhtd ON mhtd.id_tipo_documento =mp.id_tipo_documento 
WHERE nt.anio_actual_paciente  BETWEEN %s AND %s and nt.codigo_item ='%s' AND nt.tipo_diagnostico='%s' %s %s %s """ % (inicio,fin,dx,tdx, lab1, lab2, lab3)
            #print(sql)
            sql = conn.df(sql)
            ####============================ configuracion de Nombre ========================================
            lab1= "|"+row2['lab_1'] if row2['lab_1'] is not None  else ''
            lab2= "|"+row2['lab_2'] if row2['lab_2'] is not None  else ''
            lab3= "|"+row2['lab_3'] if row2['lab_3'] is not None  else ''
            head = dx.replace('.', '_')
            head = """%s|%s%s%s%s"""%(head,tdx,lab1,lab2,lab3)
            sql.rename(columns={'fecha_atencion': head}, inplace=True)
            ####================================================================================================
            head_esquema.append(head)
            ###========================================| unir con nuevo data frame |===========================
            sql = sql.set_index(['abrev_tipo_doc','numero_documento'])
            sql = sql.groupby(sql.index).tail(1)
            df_min = pd.merge(df_min, sql, on=['abrev_tipo_doc','numero_documento','genero','fecha_nacimiento','edad'], how='outer')
            df_min.reset_index() 
            #df = pd.concat([df, sql], axis=0, ignore_index=True)       
       
        # Columnas a contar
        hh="id_"+str(id_es)  
        id_col.append(hh)     
        df_min[hh] = df_min[head_esquema].count(axis=1)
        #df_min[hh] = df_min.count(axis=1) -1

        # Convertir la columna a tipo string
        condicion = f'df_min["{hh}"] {operador} {cantidad_es}'
        df_min.loc[eval(condicion), hh] = 'Si'
        #df_min.loc[df_min[hh] >= cantidad_es, hh] = 'Si'  # anterior codigo
        
        df = pd.merge(df, df_min, on=['abrev_tipo_doc','numero_documento','genero','fecha_nacimiento','edad'], how='outer')


        #print(df)
        #df.rename(columns={'col': }, inplace=True)
    df[id_col] = df[id_col].fillna(0)
    # verificar la cantidad de 'Si'
    df['total'] = df.apply(lambda row: sum(row[col] == 'Si' for col in id_col), axis=1)
    #df[id_col] = df[id_col].astype(str)
        
        

df.head(10)
      





d:\Irvin\Irvin\Python\pandas\modulos\adolescente\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\pandas\modulos\adolescente\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\pandas\modulos\adolescente\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\pandas\modulos\

,abrev_tipo_doc,numero_documento,genero,fecha_nacimiento,edad,96150_05|D,id_1,99384_02|D,id_2,Z320|D,...,id_18,D0120|D,id_20,D1330|D,id_21,D1310|D,id_22,D1110|D,id_23,total
0,CE,001127577,F,2009-02-26,14 años,2023-04-11,Si,NaN,0,NaN,...,0,NaN,0,NaN,0,NaN,0,NaN,0,1
1,CE,005777196,M,2009-10-15,13 años,2023-08-17,Si,NaN,0,NaN,...,0,NaN,0,2023-09-04,Si,2023-09-04,Si,2023-09-04,Si,8
2,CNV,32557994,F,2007-06-17,16 años,2023-07-31,Si,NaN,0,NaN,...,0,NaN,0,2023-07-31,Si,2023-07-31,Si,2023-07-31,Si,10
3,DNI,60006918,F,2006-06-28,17 años,2023-08-30,Si,2023-06-30,Si,NaN,...,0,NaN,0,NaN,0,NaN,0,NaN,0,6
4,DNI,60012381,F,2006-09-08,17 años,2023-06-23,Si,NaN,0,NaN,...,0,NaN,0,NaN,0,NaN,0,NaN,0,8
5,DNI,60014582,M,2006-09-22,17 años,2023-06-23,Si,NaN,0,NaN,...,0,NaN,0,2023-07-01,Si,2023-07-01,Si,2023-07-21,Si,8
6,DNI,60014885,M,2006-12-18,16 años,2023-04-21,Si,NaN,0,NaN,...,0,NaN,0,NaN,0,NaN,0,NaN,0,6
7,DNI,60015884,F,2006-11-05,16 años,2023-02-02,Si,NaN,0,NaN,...,0,NaN,0,NaN,0,NaN,0,NaN,0,2
8,DNI,60016379,M,2008-09-12,15 años,2023-07-13,Si,2023-08-25,Si,NaN,...,0,NaN,0,2023-06-07,Si,2023-06-07,Si,2023-06-07,Si,10
9,DNI,60016432,F,2007-05-04,16 años,2023-07-26,Si,NaN,0,2023-04-24,...,0,NaN,0,NaN,0,NaN,0,NaN,0,8


<CENTER><H1>RPT DE NEONATO<H1><CENTER>

In [6]:
data = {'abrev_tipo_doc': [],
        'numero_documento': [],
        'genero': [],
        'fecha_nacimiento': [],
        'edad': [] }
df = pd.DataFrame(data)


df_rvalidacion = consulta_rvalidacion(conn,2)

for index, row in df_rvalidacion.iterrows():
    id_p = row['id']
    inicio = row['inicio']
    fin = row['fin']

     #creamos una lista vacia para almacenas los id
    id_col= []
    df_esquema = pd.DataFrame({})
    #print(id_p)
    df_esquema=consulta_esquema(conn,id_p)

    for h, row1 in df_esquema.iterrows():
        id_es = row1['id']
        cantidad_es = row1['cantidad']

        #crea nuevo data frama pequeño 
        df_min = pd.DataFrame(data)
        #consulta SQL para jalar los codigos 
        df_codigo= consulta_codigo(conn,id_es)

        for i, row2 in df_codigo.iterrows():
            dx = row2['codigo_item']
            tdx = row2['tipo_diagnostico']
            lab1="AND nt.lab1='"+row2['lab_1']+"'" if row2['lab_1'] is not None  else ''
            lab2="AND nt.lab2='"+row2['lab_2']+"'" if row2['lab_2'] is not None  else ''
            lab3="AND nt.lab3='"+row2['lab_3']+"'" if row2['lab_3'] is not None  else '' 
            #print(1)

            sql = """ SELECT DISTINCT  mhtd.abrev_tipo_doc,mp.numero_documento, mp.genero ,mp.fecha_nacimiento  ,TO_CHAR(AGE(NOW(), fecha_nacimiento), 'YY "años" MM "meses" DD "días"') AS edad  ,  nt.fecha_atencion ,age(nt.fecha_atencion,mp.fecha_nacimiento) as tiempo_%s  FROM maestros.nominal_trama2 nt
INNER JOIN maestros.maestro_paciente mp ON mp.id_paciente =nt.id_paciente
INNER JOIN maestros.maestro_his_tipo_doc mhtd ON mhtd.id_tipo_documento =mp.id_tipo_documento  
 WHERE nt.codigo_item ='%s' AND nt.tipo_diagnostico='%s' %s  """ % (i+1,dx,tdx, lab1)
            #print(sql)
            sql = conn.df(sql)
            ####============================ configuracion de Nombre ========================================
            lab1= "|"+row2['lab_1'] if row2['lab_1'] is not None  else ''
            lab2= "|"+row2['lab_2'] if row2['lab_2'] is not None  else ''
            lab3= "|"+row2['lab_3'] if row2['lab_3'] is not None  else ''
            head = dx.replace('.', '_')
            head = """%s|%s%s%s%s"""%(head,tdx,lab1,lab2,lab3)
            sql.rename(columns={'fecha_atencion': head}, inplace=True)
            ####================================================================================================
            #  head_esquema.append(head)
            ###========================================| unir con nuevo data frame |===========================
            sql = sql.set_index(['abrev_tipo_doc','numero_documento','genero','fecha_nacimiento','edad'])
            sql = sql.groupby(sql.index).tail(1)
            
            df_min = pd.merge(df_min, sql, on=['abrev_tipo_doc','numero_documento','genero','fecha_nacimiento','edad'], how='outer')
            df_min.reset_index() 
            #df = pd.concat([df, sql], axis=0, ignore_index=True)       
       
        # Columnas a contar
        #hh="id_"+str(id_es)  
        #id_col.append(hh)     
        #df_min[hh] = df_min[head_esquema].count(axis=1)
        #df_min[hh] = df_min.count(axis=1) -1

        # Convertir la columna a tipo string
        #df_min.loc[df_min[hh] >= cantidad_es, hh] = 'Si'
        
        df = pd.merge(df, df_min, on=['abrev_tipo_doc','numero_documento','genero','fecha_nacimiento','edad'], how='outer')


        #print(df)
        #df.rename(columns={'col': }, inplace=True)
    #df[id_col] = df[id_col].fillna(0)
    # verificar la cantidad de 'Si'
    #df['total'] = df.apply(lambda row: sum(row[col] == 'Si' for col in id_col), axis=1)
    #df[id_col] = df[id_col].astype(str)
        
        

df.head(10)

d:\Irvin\Irvin\Python\pandas\modulos\adolescente\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\pandas\modulos\adolescente\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\pandas\modulos\adolescente\../..\clases\bd\conexion2.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)
d:\Irvin\Irvin\Python\pandas\modulos\

,abrev_tipo_doc,numero_documento,genero,fecha_nacimiento,edad,99381_01|D|1,tiempo_1,99381_01|D|2,tiempo_2,99381_01|D|3,tiempo_3,99381_01|D|4,tiempo_4
0,CNV,10178977,M,2022-12-24,00 años 09 meses 10 días,2023-01-04,11 days,NaN,NaT,NaN,NaT,NaN,NaT
1,CNV,10276542,M,2023-03-23,00 años 06 meses 11 días,2023-03-31,8 days,2023-04-03,11 days,2023-04-10,18 days,NaN,NaT
2,CNV,10397280,M,2023-08-04,00 años 01 meses 30 días,2023-08-09,5 days,NaN,NaT,NaN,NaT,NaN,NaT
3,CNV,25210050,F,2023-07-13,00 años 02 meses 21 días,2023-07-17,4 days,NaN,NaT,NaN,NaT,NaN,NaT
4,CNV,2528041,M,2023-01-27,00 años 08 meses 07 días,2023-01-30,3 days,2023-02-03,7 days,2023-02-10,14 days,NaN,NaT
5,CNV,2528042,F,2023-01-31,00 años 08 meses 03 días,2023-02-02,2 days,2023-02-12,12 days,2023-02-18,18 days,NaN,NaT
6,CNV,2528043,M,2023-02-03,00 años 08 meses 00 días,2023-02-06,3 days,2023-02-10,7 days,NaN,NaT,NaN,NaT
7,CNV,2528045,F,2023-02-05,00 años 07 meses 26 días,2023-02-08,3 days,NaN,NaT,NaN,NaT,NaN,NaT
8,CNV,81744177,M,2023-03-02,00 años 07 meses 01 días,2023-03-07,5 days,2023-03-10,8 days,2023-03-16,14 days,2023-03-23,21 days
9,CNV,92362355,F,2023-02-14,00 años 07 meses 17 días,2023-02-16,2 days,NaN,NaT,NaN,NaT,2023-03-09,23 days


In [7]:
#df=df.merge(paciente_df, how='inner', on=['id_paciente'])
df=df.merge(u_atencion,how='inner',on=['abrev_tipo_doc','numero_documento'])
df=df.merge(eess_df, how='inner', on=['id_establecimiento'])
df.head()

,abrev_tipo_doc,numero_documento,genero,fecha_nacimiento,edad,99381_01|D|1,tiempo_1,99381_01|D|2,tiempo_2,99381_01|D|3,tiempo_3,99381_01|D|4,tiempo_4,id_establecimiento,nombre_eess,red,microred,provincia,distrito
0,CNV,10178977,M,2022-12-24,00 años 09 meses 10 días,2023-01-04,11 days,NaN,NaT,NaN,NaT,NaN,NaT,2452,HUYRO,LA CONVENCION,MARANURA,LA CONVENCION,HUAYOPATA
1,CNV,93447420,M,2023-07-01,00 años 03 meses 02 días,2023-07-15,14 days,2023-07-22,21 days,NaN,NaT,NaN,NaT,2452,HUYRO,LA CONVENCION,MARANURA,LA CONVENCION,HUAYOPATA
2,CNV,93455220,F,2023-07-07,00 años 02 meses 27 días,2023-07-10,3 days,2023-07-14,7 days,2023-07-21,14 days,2023-07-28,21 days,2452,HUYRO,LA CONVENCION,MARANURA,LA CONVENCION,HUAYOPATA
3,CNV,93456806,M,2023-07-08,00 años 02 meses 26 días,2023-07-11,3 days,2023-07-15,7 days,2023-07-22,14 days,2023-07-29,21 days,2452,HUYRO,LA CONVENCION,MARANURA,LA CONVENCION,HUAYOPATA
4,CNV,93488099,F,2023-08-02,00 años 02 meses 01 días,2023-08-05,3 days,2023-08-09,7 days,NaN,NaT,NaN,NaT,2452,HUYRO,LA CONVENCION,MARANURA,LA CONVENCION,HUAYOPATA


In [8]:
#df.to_excel('Adolescentes.xlsx', index=False)
df.to_excel('Neonato.xlsx', index=False)


: 